<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/03-RAG_with_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [ ]:
!pip install -q llama-index==0.10.49 llama-index-llms-gemini==0.1.11 openai==1.35.3 google-generativeai==0.5.4

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(".env")

# Here we look for the OPENAI_API_KEY in the environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    # If it's not found, you can set it manually
    os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

# Get your GOOGLE_API_KEY from https://aistudio.google.com/app/apikey
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = "<YOUR_GOOGLE_KEY>"

# Load Dataset

## Download

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.

In [2]:
!curl -o ./mini-dataset.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0   772k      0 --:--:-- --:--:-- --:--:--  774k


## Read File

In [3]:
import csv

rows = []

# Load the CSV file
with open("./mini-dataset.csv", mode="r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)

    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue
            # Skip header row
        rows.append(row)

# The number of characters in the dataset.
print("number of articles:", len(rows))

number of articles: 14


# Generate Embedding

In [4]:
from llama_index.core import Document

# Convert the texts to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row[1]) for row in rows]

In [5]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding


# Build index / generate embeddings using OpenAI embedding model
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model="text-embedding-3-small"),
    transformations=[SentenceSplitter(chunk_size=768, chunk_overlap=64)],
    show_progress=True,
)

/Users/omar/Documents/ai_repos/ai-tutor-rag-system/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 14/14 [00:00<00:00, 247.39it/s]
/Users/omar/Documents/ai_repos/ai-tutor-rag-system/env/lib/python3.12/site-packages/langchain/agents/json_chat/base.py:22: SyntaxWarning: invalid escape sequence '\ '
  """Create an agent that uses JSON to format its logic, build for Chat Models.
Generating embeddings: 100%|██████████| 56/56 [00:01<00:00, 43.08it/s]


# Query Dataset

In [6]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.

from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-1.5-flash", temperature=1, max_tokens=512)

query_engine = index.as_query_engine(llm=llm)

In [7]:
response = query_engine.query("How many parameters LLaMA2 model has?")
print(response)

LLaMA 2 comes in four different sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters. 



In [8]:
response = query_engine.query("When will Llama3 will be released?")
print(response)

The context does not provide information about the release of Llama 3. 

